Follow https://www.freecodecamp.org/news/an-introduction-to-bag-of-words-and-how-to-code-it-in-python-for-nlp-282e87a9da04/ 

In [1]:
import re 
import nltk
import pandas as pd
import string

import numpy as np

# nltk.download("stopwords")

from nltk.corpus import stopwords 

In [2]:
df = pd.read_json('gs://cytora-user-wout/News_Category_Dataset_v2.json', lines=True)

### Step 1: Tokenize a sentence 

In [3]:
stopwords_set = set(stopwords.words('english'))

In [4]:
def word_extraction(sentence):
    ignore = list(stopwords_set)    
    words = re.sub("[^\w]", " ",  sentence).split()    
    cleaned_text = [w.lower() for w in words if w.lower() not in ignore]    
    return cleaned_text

In [5]:
for sen in df['headline'].sample(5):
    print(sen)
    print(word_extraction(sen))
    print()

Mark Cuban Slams U.S. Plan To Give Chinese Billionaire Monopoly On Cancer Drug
['mark', 'cuban', 'slams', 'u', 'plan', 'give', 'chinese', 'billionaire', 'monopoly', 'cancer', 'drug']

Donald Trump Sells Christmas Tree Ornament, Gets People Out Of The Holiday Spirit
['donald', 'trump', 'sells', 'christmas', 'tree', 'ornament', 'gets', 'people', 'holiday', 'spirit']

Donald Trump Can Expect A Lot Of American Taxpayers Outside The White House April 15
['donald', 'trump', 'expect', 'lot', 'american', 'taxpayers', 'outside', 'white', 'house', 'april', '15']

Get Lucky Now: 4 Simple Ways to Jump Start Your Successes in Life
['get', 'lucky', '4', 'simple', 'ways', 'jump', 'start', 'successes', 'life']

'The Birth Of A Nation' Is The Toast Of Sundance
['birth', 'nation', 'toast', 'sundance']



**TODO** Words like `isn't` and `U.S.` aren't extracted properly, maybe in case of `isn't` it doesn't matter as it will just get mapped to `isn`. 

### Step 2: Apply tokenization to all sentences
 

In [6]:
def tokenize(sentences):    
    words = []    
    for sentence in sentences:        
        w = word_extraction(sentence)        
        words.extend(w)            
        words = sorted(list(set(words)))    
    return words

In [7]:
sample2 = df['headline'].sample(2)
display(sample2)
tokenize(sample2)

113846    Skateboarders Turn Damaged Street Into Ramp Af...
37952     Everything You Need To Know Before Sunday's Go...
Name: headline, dtype: object

['california',
 'damaged',
 'earthquake',
 'everything',
 'globes',
 'golden',
 'including',
 'know',
 'need',
 'ramp',
 'skateboarders',
 'street',
 'sunday',
 'turn',
 'win']

### Step 3: Build vocabulary and generate vectors

In [8]:
def generate_bow(allsentences):        
    vocab = tokenize(allsentences)
    vector_space = pd.DataFrame()
    print(f'Done with tokenising vocabular of length: {len(vocab)}')

    for sentence in allsentences:
        words = word_extraction(sentence)
        bag_vector = np.zeros(len(vocab))
        for w in words:
            for i, word in enumerate(vocab):
                if word == w:
                    bag_vector[i] += 1
            vector_space[sentence] = np.array(bag_vector)
#     vector_space = 
    return vocab, vector_space.T

**TODO** why is the world `the` still in there? 

In [9]:
df=df.sample(10000) # REMOVE THIS STEP MemoryError when ran on all data

In [10]:
now = pd.datetime.now() 

vocab, vector_space = generate_bow(df['headline'])

diff = pd.datetime.now() - now
print(f'{diff.seconds / 60:.1f} minutes to run')

Done with tokenising vocabular of length: 14679
4.0 minutes to run


In [11]:
vector_space.columns = [str(x) for x in vocab]

In [12]:
vector_space.to_csv('data/02_bag_of_words.csv')

**TODO:** fix this

In [13]:
# vector_space[
#     (vector_space['04'] == 1) |
#     (vector_space['000'] == 1) |
#     (vector_space['1'] == 1) |
#     (vector_space['102'] == 1) 
# ].head()

In [14]:
vector_space = vector_space.reset_index()

**TODO** maybe remove vectors that represent < 6 words, this will be random anyway 

In [16]:
len(df['category'].values)

10000

# CHECK IF STILL NAMED INDEX

Otherwise change name in following cell 


In [18]:
vector_space.rename(columns={'level_0': 'head_line'}, inplace=True)

In [22]:
df.rename(columns={'headline': 'head_line'}, inplace=True)

In [26]:
df.shape

(10000, 6)

In [27]:
vector_space.shape

(9973, 14680)

Is `category` a already in the bag of words?

In [23]:
output = vector_space.merge(df[['category', 'head_line']], suffixes=('', '_target'), how='inner', on='head_line')

In [25]:
output.to_csv('data/02_bag_of_words.csv', index=False)